In [ ]:
import pandas as pd
import numpy as np
series_dt = pd.read_sql_query("SELECT id FROM cds_series_series WHERE league_id = 4", conn)
query = f"""SELECT id,start_date FROM cds_match_match WHERE series_id in {tuple(series_dt['id'])} AND start_date BETWEEN '{test_year}-{test_month}-{test_date} {start_time}:00:01' AND '{test_year}-{test_month}-{test_date} 23:59:59' """ 
match_dt = pd.read_sql_query(query, conn)
mid = match_dt['id'][0]
query = f"""SELECT id,match_id,team_id FROM cds_scorecard_scorecard WHERE match_id = {mid}"""
scorecard_scorecard_dt = pd.read_sql_query(query, conn)
query = f"""SELECT * FROM cds_match_inning_matchbatsmaninning WHERE scorecard_id in {tuple(scorecard_scorecard_dt['id'])}"""
batsman_inning_dt = pd.read_sql_query(query, conn)
query = f"""SELECT * FROM cds_match_inning_matchbowlersinning WHERE scorecard_id in {tuple(scorecard_scorecard_dt['id'])}"""
bowl_inning_dt = pd.read_sql_query(query, conn)
query = f"""SELECT * FROM cds_match_inning_matchfieldinginning WHERE scorecard_id in {tuple(scorecard_scorecard_dt['id'])}"""
fielding_inning = pd.read_sql_query(query, conn)
query = f"""SELECT * FROM cds_scorecard_matchplayers WHERE match_id = {int(match_dt['id'])} """
player_data_dt = pd.read_sql_query(query,conn)
player_data_dt = player_data_dt[{'isAllrounder','isBatsmen','isBowler','isWicketKeeper','player_id'}]
team1 = np.unique(scorecard_scorecard_dt['team_id'])[0]
team2 = np.unique(scorecard_scorecard_dt['team_id'])[1]
batsman_data_dt = batsman_inning_dt[batsman_inning_dt['scorecard_id'].isin(scorecard_scorecard_dt['id'])] 
bowler_data_dt = bowl_inning_dt[bowl_inning_dt['scorecard_id'].isin(scorecard_scorecard_dt['id'])]
query = f"""SELECT id,start_date FROM cds_match_match WHERE series_id in {tuple(series_dt['id'])} AND start_date BETWEEN '{test_year}-{test_month}-{test_date} {start_time}:00:01' AND '{test_year}-{test_month}-{test_date} 23:59:59' """ 
match2020_dt = pd.read_sql_query(query, conn)
mid2020 = match2020_dt['id'][0]
query = f"""SELECT * FROM cds_scorecard_matchplayers WHERE match_id = {mid2020} """
player_data2020_dt = pd.read_sql_query(query,conn)
playerid_team2020_dt = dict(zip(player_data2020_dt['player_id'],player_data2020_dt['team_id']))
playerid_name_dt = pd.read_sql_query("SELECT id,name FROM cds_player_player", conn)
playerid_name_dt = dict(zip(playerid_name_dt['id'],playerid_name_dt['name']))
#teamid_name = pd.read_sql_query("SELECT id,name FROM cds_team_team", conn)
#teamid_name = dict(zip(teamid_name['id'],teamid_name['name']))
batsman_data_dt = batsman_data_dt[{'balls','runs','strike_rate','fours','sixes','player_id','scorecard_id'}]
bowler_data_dt = bowler_data_dt[{'economy','wickets','player_id','scorecard_id'}]
batting_data_dt = batsman_data_dt.merge(player_data_dt, how='left', on='player_id').drop_duplicates()
bowling_data_dt = bowler_data_dt.merge(player_data_dt,how='left', on='player_id').drop_duplicates()
bowling_data_dt['fantasy_points'] = (25*bowling_data_dt['wickets'] +2*(bowling_data_dt['economy']<4)+2*(bowling_data_dt['economy']<5) +2*(bowling_data_dt['economy']<6) -2*(bowling_data_dt['economy']>9)-2*(bowling_data_dt['economy']>10) - 2*(bowling_data_dt['economy']>11)+ 8*(bowling_data_dt['wickets']>=4)+8*(bowling_data_dt['wickets']>=5))
batting_data_dt['fantasy_points'] = batting_data_dt['runs'] + (batting_data_dt['fours'] + batting_data_dt['sixes']) + 2*batting_data_dt['sixes'] + 8*(batting_data_dt['runs']>=50) + 8*(batting_data_dt['runs']>=100) - 2*(batting_data_dt['runs']==0) - 2*(batting_data_dt['balls']>=10)*(batting_data_dt['strike_rate']<70)*(batting_data_dt['strike_rate']>=60) - 4*(batting_data_dt['balls']>=10)*(batting_data_dt['strike_rate']<60)*(batting_data_dt['strike_rate']>=50) - 6*(batting_data_dt['balls']>=10)*(batting_data_dt['strike_rate']<50)
fielding_points = {}
for i in fielding_inning.index:
    if(list(fielding_points.keys()).count(fielding_inning['fielder_id'][i])==1):
        if(fielding_inning['isCaught'][i]):
            fielding_points[fielding_inning['fielder_id'][i]]+=8
        if(fielding_inning['isRunOut'][i]):
            fielding_points[fielding_inning['fielder_id'][i]]+=12
    else:
        if(fielding_inning['isCaught'][i]):
            fielding_points[fielding_inning['fielder_id'][i]] = 8
        if(fielding_inning['isRunOut'][i]):
            fielding_points[fielding_inning['fielder_id'][i]] = 12
bowling_data_dt = bowling_data_dt[{'scorecard_id','fantasy_points','player_id'}]
batting_data_dt = batting_data_dt[{'scorecard_id','fantasy_points','player_id'}]  
scorecard_scorecard_dt['scorecard_id'] = scorecard_scorecard_dt['id']
teams_dt = np.unique(np.array(list(scorecard_scorecard_dt['team_id'])))
scorecard_scorecard_dt = scorecard_scorecard_dt[{'match_id','scorecard_id'}]
bowling_data_dt = bowling_data_dt.merge(scorecard_scorecard_dt,how='left', on='scorecard_id')
batting_data_dt = batting_data_dt.merge(scorecard_scorecard_dt,how='left', on='scorecard_id')
players_data_dt = pd.concat([batting_data_dt, bowling_data_dt],0)
players_data_dt = players_data_dt[{'player_id','fantasy_points','match_id'}]
grouped_data_dt = players_data_dt.groupby(['player_id','match_id']).sum()
wicketkeeper_dt = dict(zip(player_data_dt['player_id'],player_data_dt['isWicketKeeper'].astype(int)))
bowlers_dt = dict(zip(player_data_dt['player_id'],player_data_dt['isBowler'].astype(int)))
batsmen_dt = dict(zip(player_data_dt['player_id'],player_data_dt['isBatsmen'].astype(int)))
allrounder_dt = dict(zip(player_data_dt['player_id'],player_data_dt['isAllrounder'].astype(int)))
player_team1_team2_dt = list(player_data_dt['player_id'])
fantasy_points_dt = {}
for i in player_team1_team2_dt:
    fantasy_points_dt[i]=0
j = 0
for i in grouped_data_dt.index:
    a = int(i[0])
    fantasy_points_dt[a] = fantasy_points_dt[a] + list(grouped_data_dt['fantasy_points'])[j]
    if(list(fielding_points.keys()).count(a)==1):
        fantasy_points_dt[a] = fantasy_points_dt[a]+fielding_points[a]
    j = j+1
#Sinlgle team prediction code (Mainly for dream team)
from pulp import *
prob = LpProblem("Dream_Team",LpMaximize)
fantasy_team1 = dict({i: LpVariable(cat=LpBinary,name="PLAYER1_{}".format(i)) for i in player_team1_team2_dt}) 
prob+= lpSum([ i for i in fantasy_team1.values()])== 11
objective = lpSum([float(fantasy_points_dt[i]) * fantasy_team1[i] for i in fantasy_team1])
prob += lpSum([fantasy_team1[i]*wicketkeeper_dt[i] for i in player_team1_team2_dt]) >= 1
prob += lpSum([fantasy_team1[i]*batsmen_dt[i] for i in player_team1_team2_dt]) >=3
prob += lpSum([fantasy_team1[i]*bowlers_dt[i] for i in player_team1_team2_dt]) >= 3
prob += lpSum([fantasy_team1[i]*allrounder_dt[i] for i in player_team1_team2_dt]) >=1
prob += objective
prob.solve()
#print("Dream Team for match between",teamid_name[team1],"and", teamid_name[team2],"are ")
max_score = 0
for j in range(1,2):
    #print("Team {} is".format(j))
    for i in fantasy_team1:
        if(value(fantasy_team1[i])==1):
            #print("Id",i,playerid_name[i],"having points",fantasy_points_dt[i])
            max_score = max_score+fantasy_points_dt[i]
    print(" ")      
print("Score for Dream Team is",max_score)